In [46]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import fmin_cg
import scipy.stats as st
import copy

%matplotlib inline

In [2]:
loc = r'C:\Users\c0w00f8.WMSC\Documents\Coursera\1. Machine Learning\machine-learning-ex8\ex8\ex8_movies.mat'
movies = loadmat(loc)
# y - ratings from 943 users for 1682 movies
# r - indicators, whether user j gave movie i a rating
movies_y = movies['Y']
movies_r = movies['R']
movies_y.shape

(1682, 943)

In [3]:
# avg rating for the first movie
# only considers users who gave a rating
toyStoreAvgRating = np.mean(movies_y[0, movies_r[0,:]==1])
toyStoreAvgRating

3.8783185840707963

In [4]:
loc = r'C:\Users\c0w00f8.WMSC\Documents\Coursera\1. Machine Learning\machine-learning-ex8\ex8\ex8_movieParams.mat'
params = loadmat(loc)
params_x = params['X']
params_theta = params['Theta']
# params_theta.shape - 943 * 10 - flavor of each user
# params_x.shape - 1682 * 10 - features of each movie

In [6]:
# get a subset
num_users = 4
num_movies = 5
num_features = 3

In [7]:
x_sub = params_x[:num_movies, :num_features]
theta_sub = params_theta[:num_users, :num_features]
y_sub = movies_y[:num_movies, :num_users]
r_sub = movies_r[:num_movies, :num_users]

In [10]:
r_sub

array([[1, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=uint8)

In [11]:
y_sub[r_sub==1]

array([5, 4, 3, 4, 3, 3], dtype=uint8)

In [12]:
y_sub

array([[5, 4, 0, 0],
       [3, 0, 0, 0],
       [4, 0, 0, 0],
       [3, 0, 0, 0],
       [3, 0, 0, 0]], dtype=uint8)

In [13]:
theta_sub

array([[ 0.28544362, -1.68426509,  0.26293877],
       [ 0.50501321, -0.45464846,  0.31746244],
       [-0.43191656, -0.47880449,  0.84671111],
       [ 0.72859839, -0.27189391,  0.3268436 ]])

In [14]:
x_sub

array([[ 1.0486855 , -0.40023196,  1.19411945],
       [ 0.78085123, -0.38562591,  0.52119779],
       [ 0.64150886, -0.54785385, -0.08379638],
       [ 0.45361782, -0.80021844,  0.68048129],
       [ 0.93753789,  0.1060899 ,  0.36195295]])

In [38]:
def cofiCost(x, theta, y, r):
    num_movies = x.shape[0]
    num_users = theta.shape[0]
    num_features = x.shape[1]
    
    # hx = x.dot(theta.T)
    # hx[r==0] = 0 is the same as hx = np.multiply(hx, r)
    # j = ((hx - y) ** 2).sum() / 2 works as the same
    
    j = ((np.multiply(x.dot(theta.T), r) - y) ** 2).sum() / 2
    
    return j

In [39]:
# test J w/o regulation
testCost = cofiCost(x_sub, theta_sub, y_sub, r_sub)
testCost

22.224603725685675

In [43]:
def cofiGrad(x, theta, y, r):
    num_movies = x.shape[0]
    num_users = theta.shape[0]
    num_features = x.shape[1]
    
    #j = ((np.multiply(x.dot(theta.T), r) - y) ** 2).sum() / 2
    
    x_grad = (np.multiply(x.dot(theta.T), r) - y).dot(theta)
    theta_grad = (np.multiply(x.dot(theta.T), r) - y).T.dot(x)
        
    return x_grad, theta_grad

In [47]:
def gradCheck(x, theta, y, r, eps):
    x_grad = np.zeros(x.shape)
    theta_grad = np.zeros(theta.shape)
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x_pos = copy.copy(x)
            x_neg = copy.copy(x)
            x_pos[i, j] = x[i, j] + eps
            x_neg[i, j] = x[i, j] - eps
            j_pos = cofiCost(x_pos, theta, y, r)
            j_neg = cofiCost(x_neg, theta, y, r)
            x_grad[i, j] = (j_pos - j_neg) / 2 / eps
            
    for i in range(theta.shape[0]):
        for j in range(theta.shape[1]):
            theta_pos = copy.copy(theta)
            theta_neg = copy.copy(theta)
            theta_pos[i, j] = theta[i, j] + eps
            theta_neg[i, j] = theta[i, j] - eps
            j_pos = cofiCost(x, theta_pos, y, r)
            j_neg = cofiCost(x, theta_neg, y, r)
            theta_grad[i, j] = (j_pos - j_neg) / 2 / eps

    return x_grad, theta_grad

In [53]:
testGrad = cofiGrad(x_sub, theta_sub, y_sub, r_sub)
testGrad

(array([[-2.52899165,  7.57570308, -1.89979026],
        [-0.56819597,  3.35265031, -0.52339845],
        [-0.83240713,  4.91163297, -0.76677878],
        [-0.38358278,  2.26333698, -0.35334048],
        [-0.80378006,  4.74271842, -0.74040871]]),
 array([[-10.5680202 ,   4.62776019,  -7.16004443],
        [ -3.05099006,   1.16441367,  -3.47410789],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]]))

In [52]:
testNbrGrad = gradCheck(x_sub, theta_sub, y_sub, r_sub, 0.0001)
testNbrGrad

(array([[-2.52899165,  7.57570308, -1.89979026],
        [-0.56819597,  3.35265031, -0.52339845],
        [-0.83240713,  4.91163297, -0.76677878],
        [-0.38358278,  2.26333698, -0.35334048],
        [-0.80378006,  4.74271842, -0.74040871]]),
 array([[-10.5680202 ,   4.62776019,  -7.16004443],
        [ -3.05099006,   1.16441367,  -3.47410789],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]]))

In [61]:
diff = testGrad[0] - testNbrGrad[0], testGrad[1] - testNbrGrad[1]
if ((diff[0] < 0.0001).all() == True) & ((diff[1] < 0.0001).all() == True):
    print ("passed the grad test")

passed the grad test


In [68]:
def cofiCostReg(x, theta, y, r, lam):
    
    j = ((np.multiply(x.dot(theta.T), r) - y) ** 2).sum() / 2 + (theta ** 2).sum() * lam / 2 + (x ** 2).sum() * lam / 2
                                                                    
    return j

In [63]:
testCostReg = cofiCostReg(x_sub, theta_sub, y_sub, r_sub, 1.5)
testCostReg

31.34405624427422

In [69]:
def cofiGradReg(x, theta, y, r, lam):
    
    x_grad = (np.multiply(x.dot(theta.T), r) - y).dot(theta) + lam * x
    theta_grad = (np.multiply(x.dot(theta.T), r) - y).T.dot(x) + lam * theta
        
    return x_grad, theta_grad

In [64]:
def gradCheckReg(x, theta, y, r, eps, lam):
    x_grad = np.zeros(x.shape)
    theta_grad = np.zeros(theta.shape)
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x_pos = copy.copy(x)
            x_neg = copy.copy(x)
            x_pos[i, j] = x[i, j] + eps
            x_neg[i, j] = x[i, j] - eps
            j_pos = cofiCostReg(x_pos, theta, y, r, lam)
            j_neg = cofiCostReg(x_neg, theta, y, r, lam)
            x_grad[i, j] = (j_pos - j_neg) / 2 / eps
            
    for i in range(theta.shape[0]):
        for j in range(theta.shape[1]):
            theta_pos = copy.copy(theta)
            theta_neg = copy.copy(theta)
            theta_pos[i, j] = theta[i, j] + eps
            theta_neg[i, j]b = theta[i, j] - eps
            j_pos = cofiCostReg(x, theta_pos, y, r, lam)
            j_neg = cofiCostReg(x, theta_neg, y, r, lam)
            theta_grad[i, j] = (j_pos - j_neg) / 2 / eps

    return x_grad, theta_grad

In [67]:
testGradReg = cofiGradReg(x_sub, theta_sub, y_sub, r_sub, 1.5)
testNbrGradReg = gradCheckReg(x_sub, theta_sub, y_sub, r_sub, 0.0001, 1.5)
diff = testGradReg[0] - testNbrGradReg[0], testGradReg[1] - testNbrGradReg[1]
if ((diff[0] < 0.0001).all() == True) & ((diff[1] < 0.0001).all() == True):
    print ("passed the grad test")

passed the grad test


In [81]:
def flatGrad(x, theta, y, r, lam):
    
    x_grad = (np.multiply(x.dot(theta.T), r) - y).dot(theta) + lam * x
    theta_grad = (np.multiply(x.dot(theta.T), r) - y).T.dot(x) + lam * theta
    
    grad = np.append(x_grad.flatten(), theta_grad.flatten())
    
    return grad

In [125]:
def randInit(nrow, ncol):
    epsilon = 1
    sample = np.random.uniform(-epsilon, epsilon, nrow * ncol)
    return sample.reshape(nrow, ncol)

In [88]:
def readFile(filename):
    with open(filename, 'r') as myfile:
        content = myfile.read()
    return content

In [92]:
def loadMovieList(filename):
    with open(filename, 'r') as myfile:
        content = myfile.readlines()
    movies = [x.strip('\n') for x in content]
        
    return movies

In [95]:
loc = r'C:\Users\c0w00f8.WMSC\Documents\Coursera\1. Machine Learning\machine-learning-ex8\ex8\movie_ids.txt'
movieList = loadMovieList(loc)
movieList[:10]

['1 Toy Story (1995)',
 '2 GoldenEye (1995)',
 '3 Four Rooms (1995)',
 '4 Get Shorty (1995)',
 '5 Copycat (1995)',
 '6 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '7 Twelve Monkeys (1995)',
 '8 Babe (1995)',
 '9 Dead Man Walking (1995)',
 '10 Richard III (1995)']

In [94]:
# add more ratings
myrating = np.zeros((1682, 1))
myrating.shape
myrating[0, 0] = 5
myrating[87, 0] = 4
myrating[93, 0] = 3

In [101]:
my_y = np.append(myrating, movies_y, axis=1)
rNew = np.zeros((1682, 1))
rNew[myrating > 0] = 1
my_r = np.append(rNew, movies_r, axis=1)
my_r.shape

(1682, 944)

In [140]:
my_y[:, 0]

array([5., 0., 0., ..., 0., 0., 0.])

In [105]:
myTheta = randInit(1, 10)
my_theta = np.append(myTheta, params_theta, axis=0)
my_theta.shape

(944, 10)

In [106]:
def cofiCostOpt(param, n_movies, n_users, n_features, y, r, lam):
    x = param[:n_movies*n_features]
    x = x.reshape((n_movies, n_features))
    theta = param[n_movies*n_features:]
    theta = theta.reshape((n_users, n_features))
    
    j = ((np.multiply(x.dot(theta.T), r) - y) ** 2).sum() / 2 + (theta ** 2).sum() * lam / 2 + (x ** 2).sum() * lam / 2
                                                                    
    return j

In [107]:
def flatGradOpt(param, n_movies, n_users, n_features, y, r, lam):
    x = param[:n_movies*n_features]
    x = x.reshape((n_movies, n_features))
    theta = param[n_movies*n_features:]
    theta = theta.reshape((n_users, n_features))
    
    x_grad = (np.multiply(x.dot(theta.T), r) - y).dot(theta) + lam * x
    theta_grad = (np.multiply(x.dot(theta.T), r) - y).T.dot(x) + lam * theta
    
    grad = np.append(x_grad.flatten(), theta_grad.flatten())
    
    return grad

In [129]:
def recommend(x, theta, y, r, lam):
    num_movies = x.shape[0]
    num_users = theta.shape[0]
    num_features = x.shape[1]
    
    param_init = np.append(x.flatten(), theta.flatten())
    
    myargs = num_movies, num_users, num_features, y, r, lam
    
    train = fmin_cg(cofiCostOpt, param_init, args = myargs, fprime = flatGradOpt, maxiter = 1000)
    
    x_opt = train[:num_movies*num_features]
    x_opt = x.reshape((num_movies, num_features))
    
    theta_opt = train[num_movies*num_features:]
    theta_opt = theta_opt.reshape((num_users, num_features))
    
    pred = x_opt.dot(theta_opt.T)
    
    return pred

In [144]:
def normRating(y, r):
    y_nan = copy.copy(y)
    y_nan[r==0] = np.nan
    y_mean = np.nanmean(y_nan, axis=0)
    
    y_norm = y - y_mean
    y_norm[r==0] = 0

    return y_mean, y_norm

In [147]:
testMean, testYnorm = normRating(my_y, my_r)
testYnorm.shape

(1682, 944)

In [148]:
x_init = randInit(1682, 10)
theta_init = randInit(944, 10)
testRecom = recommend(x_init, theta_init, testYnorm, my_r, 10)
testRecom

         Current function value: 40130.895407
         Iterations: 453
         Function evaluations: 678
         Gradient evaluations: 677


array([[ 0.10076441,  1.24732894, -0.26780099, ...,  0.01624832,
        -0.1709465 ,  0.08712995],
       [-0.02769504,  0.04086423, -0.3899686 , ..., -0.2160447 ,
        -0.28200503, -0.80946744],
       [-0.00775171,  1.53976749,  0.40939895, ...,  0.05720126,
        -0.2838247 , -0.78318757],
       ...,
       [-0.12122633, -0.08884285, -0.23191147, ..., -0.38688902,
        -0.39018058, -0.8560186 ],
       [-0.04688987, -1.53456327, -0.51669533, ..., -0.17316644,
         0.08746084, -0.29237494],
       [ 0.06726655,  0.5328619 ,  0.13586987, ...,  0.06960007,
        -0.02647957,  0.71746018]])

In [131]:
testRecom.shape

(1682, 944)

In [166]:
myRecom = testRecom[:,0] + testMean[0]
myRecom.max()

4.215750850366893

In [151]:
(testRecom + testMean).max()

10.070747889688985

In [167]:
myRecom = np.argsort(myRecom)[::-1]
myRecom[:5]

array([ 862,  218,  205,  612, 1138], dtype=int64)

In [172]:
top5 = list(myRecom[:5])
top5Movie = [movieList[i] for i in top5]
top5Movie

['863 Garden of Finzi-Contini, The (Giardino dei Finzi-Contini, Il) (1970)',
 '219 Nightmare on Elm Street, A (1984)',
 '206 Akira (1988)',
 '613 My Man Godfrey (1936)',
 '1139 Hackers (1995)']